# Data Retrival and Organization

Plan:
1. use API to search phyotmolecules from Datasets and get information from Pubmed to have charcteristics
2. put all this info in a csv


In [ ]:
pip install rdkit

Note: you may need to restart the kernel to use updated packages.


In [1]:
#set up imports
import pandas as pd
import requests
# RDKit for molecular descriptors
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray
import numpy as np


In [2]:
# Load the CSV file (make sure it's in your working directory or give full path)
df = pd.read_csv("CHEMICALS.csv")

# Use the 'CHEM' column for PubChem lookup
names = df['CHEM'].astype(str).str.strip().tolist()

# Clean: remove blanks and duplicates
names = [n for n in names if n]  # drop empty strings
unique_names = list(dict.fromkeys(names))  # preserves order

print(f"Loaded {len(unique_names)} unique chemical names from 'CHEM' column.")

Loaded 29585 unique chemical names from 'CHEM' column.


In [3]:
#Testing workflow for getting PubChem CIDs from informal chemical names
test_names = unique_names[:10]

import requests
import time

# Set up a requests session for performance
session = requests.Session()
results = []
not_found_count = 0

for i, name in enumerate(test_names):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{requests.utils.requote_uri(name)}/cids/JSON"
    
    try:
        response = session.get(url)
    except requests.exceptions.RequestException as e:
        print(f"[ERROR] Request failed for '{name}': {e}")
        results.append((name, None))
        not_found_count += 1
        continue

    if response.status_code != 200:
        results.append((name, None))
        not_found_count += 1
    else:
        try:
            data = response.json()
            cid = data.get('IdentifierList', {}).get('CID', [None])[0]
            results.append((name, cid))
        except Exception as e:
            print(f"[ERROR] Could not parse JSON for '{name}': {e}")
            results.append((name, None))
            not_found_count += 1

    # Throttle requests to ~5 per second
    if i % 5 == 0:
        time.sleep(1)

print(f"Done. Found CIDs for {len(results) - not_found_count} names. Not found: {not_found_count}")

Done. Found CIDs for 1 names. Not found: 9


It works! huzzah, it is okay that not all chemical names yeilded results since there are 29585 unique chemical names from 'CHEM' column

In [1]:
# Set up a requests session for performance
#  session = requests.Session()
#  results = []
#  not_found_count = 0

#   for i, name in enumerate(unique_names):
#       url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{requests.utils.requote_uri(name)}/cids/JSON"
        
#       try:
#           response = session.get(url)
#       except requests.exceptions.RequestException as e:
#           print(f"[ERROR] Request failed for '{name}': {e}")
#           results.append((name, None))
#           not_found_count += 1
#            continue

#        if response.status_code != 200:
#            results.append((name, None))
#            not_found_count += 1
#        else:
#            try:
#                data = response.json()
#                cid = data.get('IdentifierList', {}).get('CID', [None])[0]
#                results.append((name, cid))
#            except Exception as e:
#                print(f"[ERROR] Could not parse JSON for '{name}': {e}")
#                results.append((name, None))
#                not_found_count += 1

        # Throttle requests to ~5 per second
 #       if i % 5 == 0:
 #           time.sleep(1)

 #   print(f"Done. Found CIDs for {len(results) - not_found_count} names. Not found: {not_found_count}")

The code above was posted in a jupyter notebook since after 30 minutes Codespaces turns off. THis code worked though and here was the printed output: Done. Found CIDs for 12525 names. Not found: 17060

# Below is the code I put int the same Jupyter notebook to save the data to a csv

In [ ]:
# Save results to CSV
#result_df = pd.DataFrame(results, columns=["Chemical_Name", "PubChem_CID"])
result_df.to_csv("chemical_name_to_cid.csv", index=False)

# Quick preview
#result_df.head()